# 

title: Water Rights Restored to the Gila River

subtitle: The impacts of irrigation on vegetation health in the Gila
River Valley

author:

-   Elsa Culler

-   Nate Quarderer

date: last-modified

image: /img/earth-analytics/water-rights/lesson-water-rights.png

image-alt: “Dry river with dead plants turns into a stream with living
plants”

description: \|

In 2004, the Akimel O’‘otham and Tohono O’’odham tribes won a water
rights settlement in the US Supreme Court. Using satellite imagery, we
can see the effects of irrigation water on the local vegetation.

learning-goals:

-   Open raster or image data using code

-   Combine raster data and vector data to crop images to an area of
    interest

-   Summarize raster values with stastics

-   Analyze a time-series of raster images

params:

id: stars

site_name: Gila River Indian Community

event: water rights case

data_dir: gila-river

jupyter:

kernelspec:

    name: learning-portal

    language: python

    display_name: Learning Portal

# STEP 0: Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace. To save time and memory, make
sure to specify which variables you want to load.

In [1]:
%store -r ndvi_da gdf

You will also need to import any libraries you are using in this
notebook, since they won’t carry over from the previous notebook:

In [2]:
# Import libraries

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import rioxarray as rxr
import xarray as xr

import earthpy
import glob
import hvplot.pandas
import hvplot.xarray

In [3]:
# check out the dataarray
ndvi_da.head()

<xarray.Dataset> Size: 386B
Dimensions:      (date: 5, band: 1, y: 5, x: 5)
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 40B -112.3 -112.3 -112.3 -112.3 -112.3
  * y            (y) float64 40B 33.39 33.39 33.38 33.38 33.38
    spatial_ref  int64 8B 0
  * date         (date) datetime64[ns] 40B 2001-05-25 2001-06-10 ... 2001-07-28
Data variables:
    NDVI         (date, band, y, x) int16 250B 8282 6146 3796 ... 5887 5887 5359

# STEP 3: Plot NDVI

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Plot the change in NDVI spatially</div></div><div class="callout-body-container callout-body"><p>Complete the following:</p>
<ul>
<li>Select data from 2021 to 2023 (3 years after the <span
data-__quarto_custom="true" data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="1"></span>)</li>
<li>Take the temporal mean (over the <strong>date</strong>, not
spatially)</li>
<li>Get the NDVI variable (should be a DataArray, not a Dataset)</li>
<li>Repeat for the data from 2018 to 2020 (3 years before the <span
data-__quarto_custom="true" data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="2"></span>)</li>
<li>Subtract the 2018-2020 time period <strong>from</strong> the
2021-2023 time period</li>
<li>Plot the result using a <strong>diverging</strong> color map like
<code>cmap=plt.cm.PiYG</code></li>
</ul>
<p>There are different types of color maps for different types of data.
In this case, we want decreases to be a different color from increases,
so we should use a <strong>diverging</strong> color map. Check out
available colormaps in the <a
href="https://matplotlib.org/stable/tutorials/colors/colormaps.html">matplotlib
documentation</a>.</p></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-extra"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Looking for an Extra Challenge?</div></div><div class="callout-body-container callout-body"><p>For an extra challenge, add the <span data-__quarto_custom="true"
data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="3"></span> boundary to the plot.</p></div></div>

##### The text above says to select data from 2018-2020 and 2021-2023, but in class and on the website the years 2000-2012 and 2012-2020 were mentioned, so I've used that time period instead.

In [4]:
# Convert the dates to datetime format (I used chatGPT to help figure this out)
ndvi_da['date'] = pd.to_datetime(ndvi_da['date'].values, format='%Y-%m-%d')

# Select data from 2013-2020
ndvi_aft = ndvi_da.sel(date=ndvi_da['date'].dt.year.isin([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]))

# # Select data from 2021-2023
# ndvi_aft = ndvi_da.sel(date=ndvi_da['date'].dt.year.isin([2021, 2022, 2023]))


# Calculate the temporal mean
ndvi_mean_aft = ndvi_aft.mean(dim='date')
#ndvi_mean_aft.head()

# Get the NDVI variable

ndvi_post = ndvi_mean_aft['NDVI']
ndvi_post.head()


<xarray.DataArray 'NDVI' (band: 1, y: 5, x: 5)> Size: 200B
array([[[5464.16071429, 4591.07142857, 4659.89285714, 5501.66071429,
         5945.07142857],
        [4928.875     , 4263.51785714, 3559.53571429, 3694.82142857,
         3694.82142857],
        [3264.53571429, 3511.25      , 3924.03571429, 3800.30357143,
         3800.30357143],
        [3686.64285714, 4095.        , 4300.39285714, 4300.39285714,
         4212.26785714],
        [3414.71428571, 4017.53571429, 4254.83928571, 4254.83928571,
         3948.21428571]]])
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 40B -112.3 -112.3 -112.3 -112.3 -112.3
  * y            (y) float64 40B 33.39 33.39 33.38 33.38 33.38
    spatial_ref  int64 8B 0

In [5]:
# Select data from 2000-2012
ndvi_bef = ndvi_da.sel(date=ndvi_da['date'].dt.year.isin([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]))

# # Select data from 2021-2023
# ndvi_bef = ndvi_da.sel(date=ndvi_da['date'].dt.year.isin([2018, 2019, 2020]))

# Calculate the temporal mean
ndvi_mean_bef = ndvi_bef.mean(dim='date')
#ndvi_mean_bef.head()

# Get the NDVI variable

ndvi_pre = ndvi_mean_bef['NDVI']
ndvi_pre.head()


<xarray.DataArray 'NDVI' (band: 1, y: 5, x: 5)> Size: 200B
array([[[6308.        , 5094.60714286, 4500.46428571, 5274.5       ,
         5946.11904762],
        [5745.98809524, 4790.10714286, 3881.66666667, 4095.52380952,
         4095.52380952],
        [4742.5952381 , 4124.92857143, 3533.66666667, 3182.86904762,
         3182.86904762],
        [4748.21428571, 4689.66666667, 4201.98809524, 4201.98809524,
         3007.73809524],
        [3672.53571429, 4402.02380952, 4596.11904762, 4596.11904762,
         4073.16666667]]])
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 40B -112.3 -112.3 -112.3 -112.3 -112.3
  * y            (y) float64 40B 33.39 33.39 33.38 33.38 33.38
    spatial_ref  int64 8B 0

In [6]:
# Subtract before from after
ndvi_dif = ndvi_post - ndvi_pre
ndvi_dif.head()

<xarray.DataArray 'NDVI' (band: 1, y: 5, x: 5)> Size: 200B
array([[[-8.43839286e+02, -5.03535714e+02,  1.59428571e+02,
          2.27160714e+02, -1.04761905e+00],
        [-8.17113095e+02, -5.26589286e+02, -3.22130952e+02,
         -4.00702381e+02, -4.00702381e+02],
        [-1.47805952e+03, -6.13678571e+02,  3.90369048e+02,
          6.17434524e+02,  6.17434524e+02],
        [-1.06157143e+03, -5.94666667e+02,  9.84047619e+01,
          9.84047619e+01,  1.20452976e+03],
        [-2.57821429e+02, -3.84488095e+02, -3.41279762e+02,
         -3.41279762e+02, -1.24952381e+02]]])
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 40B -112.3 -112.3 -112.3 -112.3 -112.3
  * y            (y) float64 40B 33.39 33.39 33.38 33.38 33.38
    spatial_ref  int64 8B 0

In [7]:
# Select just Band 1
ndvi_dif_squeeze = ndvi_dif.sel(band=1)

# Plot the difference
(
    ndvi_dif_squeeze.hvplot(x='x', y='y', cmap=plt.cm.PiYG, geo=True, label='NEED_LABEL')
    *
    gdf.hvplot(geo=True, fill_color=None, line_color='black')
)

:Overlay
   .Image.NEED_LABEL :Image   [x,y]   (NDVI)
   .Polygons.I       :Polygons   [Longitude,Latitude]

# STEP -1: Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! Replace `var1` and `var2` with the variable you want to save,
separated by spaces.

In [8]:
%store ndvi_da ndvi_dif_squeeze gdf

Stored 'ndvi_da' (Dataset)
Stored 'ndvi_dif_squeeze' (DataArray)
Stored 'gdf' (GeoDataFrame)


Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!